In [ ]:
!pkill streamlit ## to restart live website UI
!kill $(ps aux | grep 'ngrok' | grep -v grep | awk '{print $2}')


In [ ]:
!pip install streamlit pyngrok scikit-learn pandas joblib #installing libraries


In [56]:
import streamlit as st # importing libs
import pandas as pd
import joblib


In [50]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,0,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,1,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,0,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,1,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,0,Yes,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,1,Yes,Mailed check,84.80,1990.50,0
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,1,Yes,Credit card (automatic),103.20,7362.90,0
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,0,Yes,Electronic check,29.60,346.45,0
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,0,Yes,Mailed check,74.40,306.60,1


In [ ]:
from sklearn.ensemble import RandomForestClassifier # using RF as machine learning model
from sklearn.model_selection import train_test_split #  for spliting data
import joblib

# Upload CSV
from google.colab import files
uploaded = files.upload()

# Load and preprocess
df = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv") # data was downloaded from kaggle
df = df[df["TotalCharges"] != " "]
df["TotalCharges"] = df["TotalCharges"].astype(float)
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})
df["Contract"] = df["Contract"].map({"Month-to-month": 0, "One year": 1, "Two year": 2})

X = df[["tenure", "MonthlyCharges", "Contract"]]
y = df["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Save model to file
joblib.dump(model, "churn_model.pkl")
print("✅ Model saved as churn_model.pkl")


Saving WA_Fn-UseC_-Telco-Customer-Churn.csv to WA_Fn-UseC_-Telco-Customer-Churn (1).csv
✅ Model saved as churn_model.pkl


In [51]:
X

,tenure,MonthlyCharges,Contract
0,1,29.85,0
1,34,56.95,1
2,2,53.85,0
3,45,42.30,1
4,2,70.70,0
...,...,...,...
7038,24,84.80,1
7039,72,103.20,1
7040,11,29.60,0
7041,4,74.40,0


In [54]:
y # dependent variable -  "Yes": 1, "No": 0

,Churn
0,0
1,0
2,1
3,0
4,1
...,...
7038,0
7039,0
7040,0
7041,1


In [55]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib

model = joblib.load("churn_model.pkl")

st.title("🔮 Customer Churn Predictor")

tenure = st.slider("Tenure (months)", 0, 72, 12)
monthly_charges = st.number_input("Monthly Charges", min_value=0.0, value=50.0)
contract_type = st.selectbox("Contract Type", ["Month-to-month", "One year", "Two year"])

contract_map = {"Month-to-month": 0, "One year": 1, "Two year": 2}
contract = contract_map[contract_type]

input_df = pd.DataFrame({
    'tenure': [tenure],
    'MonthlyCharges': [monthly_charges],
    'Contract': [contract]
})

if st.button("Predict"):
    prediction = model.predict(input_df)[0]
    prob = model.predict_proba(input_df)[0][1]

    if prediction == 1:
        st.error(f"⚠️ Likely to churn. Probability: {prob:.2f}")
    else:
        st.success(f"✅ Likely to stay. Probability: {1 - prob:.2f}")


Overwriting app.py


In [ ]:
from pyngrok import ngrok

# Start streamlit in background
!streamlit run app.py &>/content/logs.txt &

# Open the tunnel (this is the fixed version)
public_url = ngrok.connect("8501", "http")
print(f"🌐 Your Streamlit app is live at: {public_url}")


🌐 Your Streamlit app is live at: NgrokTunnel: "https://aa38-35-245-75-112.ngrok-free.app" -> "http://localhost:8501"


# New Section

In [ ]:
!ngrok config add-authtoken <code>

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pkill streamlit